In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Mike's code
df_path = "Resources/videos.csv"
df = pd.read_csv(df_path)

df[["publish date", "asdf"]] = df["publish_time"].str.split(pat="T", n=1, expand=True)


df[["publish time", "trash"]] = df["asdf"].str.split(pat=".", n=1, expand=True)


df["publish date"] = pd.to_datetime(df["publish date"])
df["publish_date"] = df["publish date"].dt.strftime('%m/%d/%Y')
df.head()

In [1]:
# Riley's code

In [2]:
# Testing testing 123
apple = 1
or

In [ ]:
# Jackie's code

In [ ]:
print(nothing)

In [ ]:
# Brian's code

In [ ]:
#this is my test cell to merge